In [19]:
# Install required libraries
!pip install tensorflow
!pip install transformers
!pip install nltk
!pip install rouge-score
!pip install gradio

In [20]:
import nltk
from nltk.tokenize import sent_tokenize
from transformers import T5ForConditionalGeneration, T5Tokenizer
from rouge_score import rouge_scorer
import gradio as gr

# Download necessary NLTK data
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [21]:
# Function for simple extractive summarization
def extractive_summary(text):
    sentences = sent_tokenize(text)
    # Placeholder logic for selecting top sentences
    summary = " ".join(sentences[:3])  # Select top 3 sentences as an example
    return summary

In [22]:
# Load the pre-trained model and tokenizer for T5
model_name = "t5-small"  # You can use larger models like 't5-base' or 't5-large'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Function for abstractive summarization
def abstractive_summary(text):
    input_ids = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = model.generate(input_ids, max_length=150, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [23]:
# Take user input
user_input = input("Please enter or paste the text you want summarized: ")

# Generate summaries
extractive_result = extractive_summary(user_input)
abstractive_result = abstractive_summary(user_input)

# Display results
print("\nExtractive Summary: ")
print(extractive_result)

print("\nAbstractive Summary: ")
print(abstractive_result)

Please enter or paste the text you want summarized: The crisis placed the BBC in a delicate position. On the one hand Reith was acutely aware that the government might exercise its right to commandeer the BBC at any time as a mouthpiece of the government if the BBC were to step out of line, but on the other he was anxious to maintain public trust by appearing to be acting independently. The government was divided on how to handle the BBC, but ended up trusting Reith, whose opposition to the strike mirrored the PM's own. Although Winston Churchill in particular wanted to commandeer the BBC to use it "to the best possible advantage", Reith wrote that Stanley Baldwin's government wanted to be able to say "that they did not commandeer [the BBC], but they know that they can trust us not to be really impartial".

Extractive Summary: 
The crisis placed the BBC in a delicate position. On the one hand Reith was acutely aware that the government might exercise its right to commandeer the BBC at 

In [24]:
# Function to evaluate and compare summaries using ROUGE scores
def evaluate_summaries(reference_text, extractive_summary, abstractive_summary):
    # Initialize ROUGE scorer
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

    # Evaluate extractive summary
    extractive_scores = scorer.score(reference_text, extractive_summary)

    # Evaluate abstractive summary
    abstractive_scores = scorer.score(reference_text, abstractive_summary)

    # Collect evaluation results
    result = "\nROUGE Scores for Extractive Summary:\n"
    for metric, score in extractive_scores.items():
        result += f"{metric}: Precision: {score.precision:.2f}, Recall: {score.recall:.2f}, F1 Score: {score.fmeasure:.2f}\n"

    result += "\nROUGE Scores for Abstractive Summary:\n"
    for metric, score in abstractive_scores.items():
        result += f"{metric}: Precision: {score.precision:.2f}, Recall: {score.recall:.2f}, F1 Score: {score.fmeasure:.2f}\n"

    return result

In [25]:
# Function for Gradio interface
def summarize_and_evaluate(user_text):
    extractive = extractive_summary(user_text)
    abstractive = abstractive_summary(user_text)
    evaluation_results = evaluate_summaries(user_text, extractive, abstractive)
    return f"Extractive Summary:\n{extractive}\n\nAbstractive Summary:\n{abstractive}\n\n{evaluation_results}"

In [26]:
# Create and launch the Gradio interface
interface = gr.Interface(
    fn=summarize_and_evaluate,
    inputs="textbox",
    outputs="text",
    title="Text Summarization Tool",
    description="Enter text to see both extractive and abstractive summaries, along with ROUGE score evaluations."
)

interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f3c629c2a005e5a56d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
